In [34]:
import tensorflow as tf

In [35]:
import autokeras as ak

In [36]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

df = pd.read_csv('data/experimental_df.csv')


In [37]:
X_train = df.drop(columns=['Winner'])
X_train = X_train.drop(columns=['Unnamed: 0'])
X_train = X_train.drop(columns=['1Quantity'])
X_train = X_train.drop(columns=['2Quantity'])
y_train = df['Winner'].astype(int)

In [38]:
X_train.columns

Index(['ball_control', 'Dribbling', 'Marking', 'Slide_Tackle', 'Stand_Tackle',
       'Aggression', 'Reactions', 'Att_Position', 'Interceptions', 'Vision',
       'Short_Pass', 'Long_Pass', 'Acceleration', 'Stamina', 'Strength',
       'Balance', 'Sprint_Speed', 'Agility', 'Jumping', 'Heading',
       'Shot_Power', 'Finishing', 'Long_Shots', 'Curve', 'FK_Acc', 'Penalties',
       'Volleys', 'GK_Positioning', 'GK_Diving', 'GK_Handling', 'GK_Kicking',
       'GK_Reflexes', 'Height', 'Weight', 'Age'],
      dtype='object')

In [39]:
clf = ak.StructuredDataClassifier(overwrite=True, max_trials=200, metrics='accuracy')

clf.fit(X_train, y_train, validation_split=0.15, epochs=30)
#clf.fit(X_train, y_train, epochs=25)

Trial 57 Complete [00h 00m 03s]
val_accuracy: 0.550000011920929

Best val_accuracy So Far: 0.699999988079071
Total elapsed time: 00h 02m 48s
INFO:tensorflow:Oracle triggered exit
Epoch 1/30
11/11 [==============================] - 0s 3ms/step - loss: 1.2050 - accuracy: 0.3941
Epoch 2/30
11/11 [==============================] - 0s 3ms/step - loss: 1.0385 - accuracy: 0.4853
Epoch 3/30
11/11 [==============================] - 0s 3ms/step - loss: 0.9836 - accuracy: 0.5265
Epoch 4/30
11/11 [==============================] - 0s 3ms/step - loss: 0.9257 - accuracy: 0.5618
Epoch 5/30
11/11 [==============================] - 0s 3ms/step - loss: 0.8536 - accuracy: 0.6029
Epoch 6/30
11/11 [==============================] - 0s 3ms/step - loss: 0.8367 - accuracy: 0.6412
Epoch 7/30
11/11 [==============================] - 0s 3ms/step - loss: 0.8208 - accuracy: 0.6441
Epoch 8/30
11/11 [==============================] - 0s 3ms/step - loss: 0.6683 - accuracy: 0.7353
Epoch 9/30
11/11 [===================

custom_f1 from https://neptune.ai/blog/implementing-the-macro-f1-score-in-keras

In [40]:
from tensorflow.keras import backend as K

def custom_f1(y_true, y_pred):
    def recall_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Positives = K.sum(K.round(K.clip(y_true, 0, 1)))

        recall = TP / (Positives+K.epsilon())
        return recall


    def precision_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Pred_Positives = K.sum(K.round(K.clip(y_pred, 0, 1)))

        precision = TP / (Pred_Positives+K.epsilon())
        return precision

    precision, recall = precision_m(y_true, y_pred), recall_m(y_true, y_pred)

    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [41]:
mmm = ak.StructuredDataClassifier(overwrite=True, max_trials=200, metrics=[custom_f1, 'accuracy'])

mmm.fit(X_train, y_train, validation_split=0.15, epochs=30)
#mmm.fit(X_train, y_train, epochs=25)

Trial 121 Complete [00h 00m 03s]
val_accuracy: 0.6000000238418579

Best val_accuracy So Far: 0.75
Total elapsed time: 00h 06m 35s
INFO:tensorflow:Oracle triggered exit
Epoch 1/30
11/11 [==============================] - 1s 3ms/step - loss: 3.8501 - custom_f1: 0.3247 - accuracy: 0.3294
Epoch 2/30
11/11 [==============================] - 0s 3ms/step - loss: 1.4953 - custom_f1: 0.3776 - accuracy: 0.4353
Epoch 3/30
11/11 [==============================] - 0s 4ms/step - loss: 1.0351 - custom_f1: 0.4302 - accuracy: 0.5500
Epoch 4/30
11/11 [==============================] - 0s 4ms/step - loss: 0.9397 - custom_f1: 0.5143 - accuracy: 0.5706
Epoch 5/30
11/11 [==============================] - 0s 3ms/step - loss: 0.9399 - custom_f1: 0.5592 - accuracy: 0.5912
Epoch 6/30
11/11 [==============================] - 0s 4ms/step - loss: 0.9074 - custom_f1: 0.5573 - accuracy: 0.6118
Epoch 7/30
11/11 [==============================] - 0s 4ms/step - loss: 0.7362 - custom_f1: 0.6639 - accuracy: 0.7059
Epoch 

In [49]:
import keras
keras.utils.get_custom_objects()['custom_f1'] = custom_f1

In [50]:
model = mmm.export_model()

In [51]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 35)]              0         
                                                                 
 multi_category_encoding (Mu  (None, 35)               0         
 ltiCategoryEncoding)                                            
                                                                 
 dense (Dense)               (None, 256)               9216      
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1024)              263168

In [52]:
model.save('autokeras_model')

INFO:tensorflow:Assets written to: autokeras_model/assets
